In [1]:
import os
import random
import json

import cloudpickle
import pandas as pd
import numpy as np
import requests


In [20]:
INPUT_DATA = "static/data/inputs.csv"
GEN_SIZE = 5000
FEATURES = ["sepal-length", "sepal-width", "petal-length", "petal-width"]
LABEL = "species"
IRIS_SERVING_URL = "http://localhost:3000"

raw_data = pd.read_csv(INPUT_DATA)

# Helper Functions

In [21]:
def _writeCSVToTextFile(examples):
    data_location = INPUT_DATA
    with open(data_location, "w") as writer:
        for example in examples:
            writer.write(example + "\n")
    return data_location


def load_logreg():
    with open("logreg.pkl", "rb") as logreg_io:
        model = cloudpickle.load(logreg_io)
    with open("scaler.pkl", "rb") as scaler_io:
        scaler = cloudpickle.load(scaler_io)
    return scaler, model


def gen_data_drift(drift_mean=0.5, drift_std=0.2):
    inputs = raw_data[FEATURES]
    index = np.random.choice(range(inputs.shape[0]), GEN_SIZE)
    inputs = inputs.iloc[index]
    noises = np.random.normal(loc=drift_mean, scale=drift_std, size=inputs.shape)
    inputs = inputs + noises
    return inputs


# Request for inference

In [67]:
DRIFT_MEAN = [0, 0, 0, 0]
# DRIFT_STD = [0.6, 0.5, 1.0, 1.0]
DRIFT_STD = [0, 0, 0, 0]
input_data = gen_data_drift(drift_mean=DRIFT_MEAN, drift_std=DRIFT_STD)
response = requests.post(
    f"{IRIS_SERVING_URL}/inference",
    json=input_data.to_json(orient="split"),
    params={"prediction_key": "classes"},
)
response_json = json.loads(response.text)
anomaly = response_json["Anomaly"]
prediction = response_json["classes"]
print(prediction)

['Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa', 'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-virginica', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicol

# Label data and send back to server

In [68]:
scaler, logreg = load_logreg()
label_data = input_data.copy()
label_data['species'] = logreg.predict(scaler.transform(input_data))
response = requests.post(
    f"{IRIS_SERVING_URL}/upload_label_data",
    json=label_data.to_json(orient="split")
)
print(response.text)

Label data uploaded.
